In [141]:
import serial, struct
ser = serial.Serial('COM3', 115200) 


In [159]:
r = 255
g = 100
b = 50
laser_x = 1000
laser_y = 2000
audio_l = 300
audio_r = 400
timestamp = 10000
payload = struct.pack('<BBBHHHHQ', r, g, b, laser_x, laser_y, audio_l, audio_r, timestamp)

for i in range(10):
    ser.write(payload)


In [ ]:
ser.close()

In [206]:
16*8

128

In [177]:
19*107/.0016

1270625.0

In [203]:
import numpy as np
np.log2(10000000/20 * 3600*24*25), 8*5

(39.974168451037094, 40)

In [2]:
(1117780 -  462420)/128

5120.0